## Illustration of the simple wrapper for uvspec_etc 

First, import the wrapper function for obtaining UVI SNR from basic inputs 

In [ ]:
from syotools.wrappers.uvspec import uvspec_snr, uvspec_exptime

In [ ]:
help(uvspec_snr)

In [ ]:
wave120, snr120, uvi = uvspec_snr('EAC1', 'G120M', 'flat', 19, 3.) 
wave150, snr150, uvi = uvspec_snr('EAC1', 'G150M', 'flat', 19, 3.) 
wave180, snr180, uvi = uvspec_snr('EAC1', 'G180M', 'flat', 19, 3.) 

wave300, snr300, uvi = uvspec_snr('EAC1', 'G300M', 'flat', 19, 1.) 

In [ ]:
import matplotlib.pyplot as plt 

plt.plot(wave120, snr120, label='G120M')
plt.plot(wave150, snr150, label='G150M') 
plt.plot(wave180, snr180, label='G180M') 
plt.plot(wave300, snr300, label='G300M') 

plt.legend() 
plt.xlabel('Wavelength [Angstrom]') 
plt.ylabel('SNR per pixel') 

In [ ]:
def uvspec_exptime(telescope, mode, template, fuvmag, snr_goal): 
    
    from syotools.models import Telescope, Spectrograph
    from syotools.utils.jsonunit import str_jsunit
    from syotools.spectra import SpectralLibrary
    import astropy.units as u

    # create the basic objects 
    uvi, tel = Spectrograph(), Telescope() 
    tel.set_from_json(telescope)
    tel.add_spectrograph(uvi)
    uvi_exp = uvi.create_exposure()
    uvi.mode = mode

    uvi_exp.sed_id = template
    uvi_exp.renorm_sed(fuvmag * u.ABmag, bandpass='galex,fuv')

    #Print the current template & mode
    print("Current SED template: {}".format(SpectralLibrary[uvi_exp.sed_id])) 
    print("Current grating mode: {}".format(uvi.descriptions[uvi.mode])) 
    print("Current exposure time: {} hours\n".format(uvi_exp.exptime[1]['value'][0])) 
    
    lumos_template_codes = ['flam', 'qso', 's99', 'o5v', 'g2v', 'g191b2b', 'gd71', 'gd153', 'ctts', 
                        'mdwarf', 'orion', 'nodust', 'ebv6', 'hi1hei1', 'hi0hei1']
    available_lumos_templates = [SpectralLibrary[tc] for tc in lumos_template_codes]

    uvi_exp._snr_goal= snr_goal * (u.ct)**0.5 / (u.pix)**0.5

    snr = uvi_exp.recover('exptime')
    uvi_exp.unknown = 'exptime' #< --- this triggers the _update_exptime function in the SpectrographicExposure exposure object 

    #uvi_exp.enable()
    uvi_sed, uvi_exptime = uvi_exp.recover('sed', 'exptime')

    wave, exptime =  uvi.wave[1]['value'], uvi_exp.exptime[1]['value']

    return wave, exptime, uvi 



In [ ]:
#                                       tel.    mode. template fuvmag  snrgoal 
wave120, exp120, uvi = uvspec_exptime('EAC1', 'G120M', 'flat', 19,      10.) 
wave150, exp150, uvi = uvspec_exptime('EAC1', 'G150M', 'flat', 19,      10.) 
wave180, exp180, uvi = uvspec_exptime('EAC1', 'G180M', 'flat', 19,      10.) 
wave300, exp300, uvi = uvspec_exptime('EAC1', 'G300M', 'flat', 19,      10.) 

#### NOTE: the uvspec_exptime function returns the time it would take to reach the snr_goal value at each pixel.  These SNR results cannot be obtained all at the same time! The system response varies with wavelength, but all wavelengths are observed at once for any given grating choice. So, use this to determine how long it would take to achive your SNR goal at your favorite wavelength, and disregard the other wavelengths in that trace.


In [ ]:
plt.plot(wave120, exp120, label='G120M')
plt.plot(wave150, exp150, label='G150M') 
plt.plot(wave180, exp180, label='G180M') 
plt.plot(wave300, exp300, label='G300M') 

plt.legend() 
plt.ylim(0, 10000)
plt.xlabel('Wavelength [Angstrom]') 
_ = plt.ylabel('SNR per pixel') 